In [1]:
# So can get yadlt
import sys
sys.path.append('/usr2/mamille2/nn4nlp-scripts/mm_dbm/Deep-Learning-TensorFlow/')

import pandas as pd
from collections import defaultdict
import numpy as np
import tensorflow as tf
import inspect

import dbm
import config
import rbm

# Pretraining

In [78]:
dataset = {}

# for f in ['train', 'valid']:
for f in ['train']:
    dataset[f] = np.load("/usr2/mamille2/twitter/data/huang2016_data/tags_{}.npy".format(f))

# D = np.sum(np.vstack((dataset['train'], dataset['valid'])), axis=1)  # length of each document
D = np.sum(dataset['train'], axis=1)  # length of each document

In [79]:
model = dbm.DBM(
    main_dir="/usr2/mamille2/twitter/data/huang2016_data/twitter_tags_rbm", do_pretrain=True, layers=[1024, 1024],
    models_dir=config.models_dir, data_dir=config.data_dir, summary_dir=config.summary_dir,
    learning_rate=[0.001, 0.001], momentum=0.9, num_epochs=[1, 1], batch_size=[64, 64],
    stddev=0.1, verbose=1, gibbs_k=[1, 1], model_name="twitter_tags_dbm",
    finetune_learning_rate=0.01, finetune_enc_act_func=[tf.nn.sigmoid, tf.nn.sigmoid],
    finetune_dec_act_func=[tf.nn.sigmoid, tf.nn.sigmoid], finetune_num_epochs=1, finetune_batch_size=128,
    finetune_opt='momentum', finetune_loss_func="mean_squared", finetune_dropout=0.5, noise=["rsm", "bin"],
)

Creating /usr2/mamille2/twitter/data/mm_dbm/model/stored_models/ directory to save/restore models
Creating /usr2/mamille2/twitter/data/mm_dbm/model/data/ directory to save model generated data
Creating /usr2/mamille2/twitter/data/mm_dbm/model/logs/ directory to save tensorboard logs
Creating /usr2/mamille2/twitter/data/mm_dbm/model/stored_models/rbm-1 directory to save/restore models
Creating /usr2/mamille2/twitter/data/mm_dbm/model/data/rbm-1 directory to save model generated data
Creating /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-1 directory to save tensorboard logs
Creating /usr2/mamille2/twitter/data/mm_dbm/model/stored_models/rbm-2 directory to save/restore models
Creating /usr2/mamille2/twitter/data/mm_dbm/model/data/rbm-2 directory to save model generated data
Creating /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-2 directory to save tensorboard logs


In [80]:
# Initialize documents lengths
for i, _ in enumerate(model.rbms):
    model.rbms[i].D = D

In [60]:
print(len(dataset['train']))
print(len(dataset['valid']))

215118
26039


In [81]:
# Pretraining phase 
# model.pretrain(dataset['train'][:500], dataset['valid'][500:1000])
model.pretrain(dataset['train'][:10000], dataset['train'][10000:20000])

Training layer 1...
Tensorboard logs dir for this run is /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-1/run10
Reconstruction loss at step 0: 0.012994
Training layer 2...
Tensorboard logs dir for this run is /usr2/mamille2/twitter/data/mm_dbm/model/logs/rbm-2/run10
Reconstruction loss at step 0: 0.0155692


(array([[ 0.78262526,  0.24725132,  0.2698831 , ...,  0.39486814,
          0.45765743,  0.03806502],
        [ 0.78262526,  0.24725132,  0.2698831 , ...,  0.39486814,
          0.45765743,  0.03806502],
        [ 0.77756256,  0.24815096,  0.28044698, ...,  0.39678276,
          0.44103429,  0.03584903],
        ..., 
        [ 0.78261638,  0.24571618,  0.27131805, ...,  0.43171808,
          0.44519871,  0.04281552],
        [ 0.78262526,  0.24725132,  0.2698831 , ...,  0.39486814,
          0.45765743,  0.03806502],
        [ 0.78262526,  0.24725132,  0.2698831 , ...,  0.39486814,
          0.45765743,  0.03806502]], dtype=float32),
 array([[ 0.78535336,  0.26702645,  0.27771628, ...,  0.34255692,
          0.43727925,  0.03904784],
        [ 0.78262526,  0.24725132,  0.2698831 , ...,  0.39486814,
          0.45765743,  0.03806502],
        [ 0.78703976,  0.27790949,  0.30201226, ...,  0.38000956,
          0.43650389,  0.03872642],
        ..., 
        [ 0.78262526,  0.24725132,  0

In [83]:
# Fit and save the txt-DBM parameters 
# I put save_dbm_text_params as a quick hack to save the parameters of this dbm as a numpy array

# takes args train_set, train_ref, validation_set, validation_ref -- unsupervised, so just use more of the dataset

train_split = len(dataset['train'])//4
# valid_split = len(dataset['valid'])//4

# model.fit(dataset['train'][:500], dataset['train'][500:1000], dataset['valid'][:500], dataset['valid'][500:1000])
# model.fit(dataset['train'][:train_split], dataset['train'][train_split:], dataset['valid'][:valid_split], dataset['valid'][valid_split:-1])
model.fit(dataset['train'][:train_split], dataset['train'][train_split:2*train_split], 
          dataset['train'][2*train_split:3*train_split], dataset['train'][3*train_split:-2])
# model.fit(dataset['train'][:1000], dataset['train'][1000:2000], dataset['valid'][:500], dataset['valid'][500:1000])
# model.fit(dataset['train'][:1000], dataset['train'][1000:2000], dataset['valid'][:1000], dataset['valid'][1000:2000])

Tensorboard logs dir for this run is /usr2/mamille2/twitter/data/mm_dbm/model/logs/run9
Reconstruction loss at step 0: 0.430305


In [84]:
# Save out model params
outpath = '/usr2/mamille2/twitter/data/mm_dbm/model/tag_dbms_layer_{}_{}.npy'

for i in range(len(model.rbms)):
    for p in ['W', 'bh_', 'bv_']: # not sure whether uses bh or bv
        np.save(outpath.format(i, p), model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])[p])

In [16]:
model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])['W'].shape

(5, 1024)

In [17]:
model.rbms[1].get_model_parameters(graph=model.rbm_graphs[1])['W'].shape

(1024, 1024)

In [18]:
model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])['bh_'].shape

(1024,)

In [19]:
model.rbms[0].get_model_parameters(graph=model.rbm_graphs[0])['bv_'].shape

(5,)

# Format tweet hashtags

In [19]:
data = {}

for f in ['train', 'valid', 'test']:
    data[f] = pd.read_pickle('/usr2/mamille2/twitter/data/huang2016_data/huang2016_{}.pkl'.format(f))
    
data['train'].columns

Index(['tweet_id', 'created_at', 'in_reply_to_status_id', 'lang',
       'retweet_count', 'user_id', 'user_screen_name', 'user_name', 'text',
       'text_no_tags', 'tags'],
      dtype='object')

In [43]:
# Load tag vocabulary

with open('/usr2/mamille2/twitter/data/huang2016_data/hashtag_vocab.txt') as f:
    tags = f.read().splitlines()

tag_vocab_len = len(tags)
tag_vocab_len

3890

In [32]:
# Create vocab indices dict
tag2idx = defaultdict(int)

for t in tags:
    tag2idx[t] = tags.index(t)
    
len(tag2idx)

3890

In [35]:
# def tags2indices(tags, max_len):
def tags2indices(tags):
    """ Converts tags to indices """
    inds = [tag2idx[t] for t in tags]
#     padding = [0] * (max_len - len(tags))
    return inds

In [17]:
# Convert from tag indices to one-hot vectors
def inds2hot(tag_inds, len_vocab):
    """ Converts tags from indices to one-hot vectors """
    
    onehot = np.zeros(len_vocab)
    for l in tag_inds:
        onehot[l] = 1
        
    return onehot

In [36]:
# Covert from tags to one-hot vectors
def tags2hot(tags, vocab_len):
    return inds2hot(tags2indices(tags), vocab_len)

In [47]:
# Convert dataset to tag indices
tag_vecs = {}

for f in ['train', 'valid', 'test']:
    tag_vecs[f] = np.array(data[f]['tags'].map(lambda x: tags2hot(x, tag_vocab_len)).tolist())
    
    np.save('/usr2/mamille2/twitter/data/huang2016_data/tags_{}.npy'.format(f), tag_vecs[f])

In [45]:
tag_vecs['train']

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [31]:
# Check tag vocabulary
vocab = set([t for l in data['train']['tags'].tolist() for t in l])
len(vocab)

37644

In [15]:
# Find max number of tags
len_tags = train_data['tags'].map(lambda x: len(x))
max(len_tags)

# Find max number of tags
len_tags = valid_data['tags'].map(lambda x: len(x))
max(len_tags)

# Find max number of tags
len_tags = test_data['tags'].map(lambda x: len(x))
max(len_tags)

4

In [15]:
test = tags2indices(['coaching', 'walking'], MAX_LEN)
test

[1, 2]

In [16]:
testvec = inds2hot(test, tag_vocab_len)
testvec[:10]

array([ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])